In [265]:
import typing
from functools import reduce
import numpy as np
from math import log, exp


# Constucts a callable which computes the response
# initialize with MLE theta
def response(theta: np.array) -> int:
    def _exp(x_i: np.array):
        return 1 if 1/(1 + exp(-np.dot(theta.T, x_i))) > 0.5 else 0
    return _exp

# Constucts a callable for the log-liklihood function
# initialize with y and x
def log_lik(y: np.array, x: np.ndarray) -> float:
    def _LL(theta: np.array)-> float:
        def _LL_log_exp(y_i: float, x_i: np.array)-> float:
            return y_i *log(1/(1 + exp(-np.dot(theta.T, x_i))))\
                + (1-y_i)*log(1/(1 + exp(np.dot(theta.T, x_i))))
        return np.sum(np.array(list(map(_LL_log_exp, y, x))))
    return _LL  


# Constucts a callable for the gradient of the 
# log-liklihood for use in grad_desc
# initialize with y and x
def log_lik_grad(y: np.array, x: np.ndarray) -> np.array:
    def _LL_grad(theta: np.array)-> np.array:
        def _LL_exp(y_i: float, x_i: np.array)-> np.array:
            return (y_i - 1/(1 +exp(-np.dot(theta.T, x_i)))) * x_i
        return np.array(list(reduce(np.add, map(_LL_exp, y, x)))) 
         
    return _LL_grad  

# Constucts a callable for the Fisher matrix of the 
# log-liklihood for use in grad_desc
# initialize with y and x
def log_lik_Fisher(x: np.ndarray) -> np.ndarray:
    def _LL_Fisher(theta: np.array)-> np.ndarray:
        def _LL_exp_2(x_i: np.array)-> np.ndarray:
            return (exp(-np.dot(theta.T,x_i)))/\
                    ((1+exp(-np.dot(theta.T,x_i)))**2)\
                    * np.outer(x_i, x_i.T)
        return np.array(list(reduce(np.add, map(_LL_exp_2,x)))) 
         
    return _LL_Fisher

# Our implementation of gradient ascent 
def grad_asc(grad_f: callable, func: callable, y: np.array, x: np.ndarray,\
              step_size: float=1e-6, tol: float=1e-4, max_iter=2500000)-> np.array:
    # random initilaizer
    theta = np.random.randn(len(x[0]))
    prev_theta = np.zeros(len(x[0]))
    err = np.linalg.norm(theta - prev_theta)
    #grad_f must be callable in variable we want to optimize
    nabla = grad_f(y,x)
    f = func(y,x)
    current_iter = 0
    while (err > tol and current_iter < max_iter):
        # update theta
        prev_theta = theta
        theta = theta + step_size * nabla(theta)
        err = abs(f(theta) - f(prev_theta))
        current_iter +=1
    return theta 

In [165]:
import pandas as pd
df = pd.read_csv("C:/Users/baeza/OneDrive/Desktop/CS 760 Fall 2020/titanic_data.csv").astype(float)

In [166]:
# survived
y = df.iloc[:,0].astype(float).to_numpy()

In [167]:
x_temp = df.iloc[:,1:].astype(float).to_numpy()
s = np.ones((887,1))
x = np.concatenate((x_temp,s), axis =1)

In [175]:
%%time
theta_hat = grad_asc(log_lik_grad,log_lik, y, x, step_size=2.5e-6, tol=5e-6, max_iter=2.5e6)
print(f"theta_hat is {theta_hat}")
print(f"log_lik(theta_hat) is {log_lik(y,x)(theta_hat)}")
print(f"log_lik_Fisher is \n {log_lik_Fisher(x)(theta_hat)}")

theta_hat is [-1.08264619  2.74309386 -0.03932736 -0.38888148 -0.1106569   0.00358926
  2.17297116]
log_lik(theta_hat) is -390.73359320925783
log_lik_Fisher is 
 [[7.12057248e+02 1.25190264e+02 7.47353297e+03 1.44608277e+02
  1.22718926e+02 6.13943112e+03 2.79910046e+02]
 [1.25190264e+02 5.02962423e+01 1.38224277e+03 3.82204493e+01
  3.52309268e+01 1.40207876e+03 5.02962423e+01]
 [7.47353297e+03 1.38224277e+03 1.32625916e+05 1.32885723e+03
  1.27786136e+03 1.25971550e+05 3.68471054e+03]
 [1.44608277e+02 3.82204493e+01 1.32885723e+03 1.37010256e+02
  6.06420378e+01 2.67212436e+03 6.07943669e+01]
 [1.22718926e+02 3.52309268e+01 1.27786136e+03 6.06420378e+01
  1.12804430e+02 2.52073174e+03 5.12849796e+01]
 [6.13943112e+03 1.40207876e+03 1.25971550e+05 2.67212436e+03
  2.52073174e+03 3.64337514e+05 3.83484750e+03]
 [2.79910046e+02 5.02962423e+01 3.68471054e+03 6.07943669e+01
  5.12849796e+01 3.83484750e+03 1.25803068e+02]]
Wall time: 42min 41s


In [266]:
fisher_inv = np.linalg.inv(log_lik_Fisher(x)(theta_hat))
print(f"log_lik_Fisher inverse is \n {fisher_inv}")

log_lik_Fisher inverse is 
 [[ 2.10014540e-02 -6.26489237e-03  4.82633027e-04  2.30427468e-04
  -1.74234958e-03  1.79143845e-04 -6.32212092e-02]
 [-6.26489237e-03  3.93631834e-02 -1.45990201e-04 -3.59299176e-03
  -4.65700196e-03 -1.76918377e-06  6.16654432e-03]
 [ 4.82633027e-04 -1.45990201e-04  5.78115350e-05  2.03965965e-04
   3.34449821e-05  5.96511197e-07 -2.83914049e-03]
 [ 2.30427468e-04 -3.59299176e-03  2.03965965e-04  1.19523829e-02
  -3.21555909e-03 -3.80434625e-05 -8.35573670e-03]
 [-1.74234958e-03 -4.65700196e-03  3.34449821e-05 -3.21555909e-03
   1.37932075e-02 -6.38906070e-05  2.63753524e-03]
 [ 1.79143845e-04 -1.76918377e-06  5.96511197e-07 -3.80434625e-05
  -6.38906070e-05  6.11319851e-06 -5.57274782e-04]
 [-6.32212092e-02  6.16654432e-03 -2.83914049e-03 -8.35573670e-03
   2.63753524e-03 -5.57274782e-04  2.49256943e-01]]


In [267]:
from scipy.stats import norm, chi2
from math import sqrt

#Feature vector we create ourselves
x_new = np.array([3, 1, 27, 0, 1, 7.25, 1])

print(f"The response for x_new is {response(theta_hat)(x_new)}")

omega_hat = np.dot(theta_hat.T, x_new)

mod_var = np.dot(x_new.T, np.dot(fisher_inv, x_new))  
                            
tau = norm.ppf(0.975, loc = 0, scale = sqrt(mod_var) )

print(f"tau = {tau}")
print(f"the 95 percent confidence interval for omega_hat is \n ({omega_hat - tau},{omega_hat + tau})")

theta1_hat, theta2_hat ,theta3_hat = theta_hat[0], theta_hat[1], theta_hat[2]
theta4_hat, theta5_hat ,theta6_hat = theta_hat[3], theta_hat[4], theta_hat[5]

v1, v2, v3, = fisher_inv[0][0], fisher_inv[1][1], fisher_inv[2][2]
v4, v5, v6, = fisher_inv[3][3], fisher_inv[4][4], fisher_inv[5][5] 
phi_inv = chi2.ppf(0.95, 1)
p1 = chi2.sf((theta1_hat**2)/(v1**2), 1)
p2 = chi2.sf((theta2_hat**2)/(v2**2), 1)
p3 = chi2.sf((theta3_hat**2)/(v3**2), 1)
p4 = chi2.sf((theta4_hat**2)/(v4**2), 1)
p5 = chi2.sf((theta5_hat**2)/(v5**2), 1)
p6 = chi2.sf((theta6_hat**2)/(v6**2), 1)

exp1 = "" if theta1_hat**2 > (v1**2) * phi_inv else "not " 
exp2 = "" if theta2_hat**2 > (v2**2) * phi_inv else "not "
exp3 = "" if theta3_hat**2 > (v3**2) * phi_inv else "not " 
exp4 = "" if theta4_hat**2 > (v4**2) * phi_inv else "not "
exp5 = "" if theta5_hat**2 > (v5**2) * phi_inv else "not " 
exp6 = "" if theta6_hat**2 > (v6**2) * phi_inv else "not "

print(f"Passenger class is {exp1}significant: p-value {p1}")
print(f"Gender is {exp2}significant: p-value {p2}")
print(f"Age is {exp3}significant: p-value {p3}") 
print(f"Siblings/Spouses is {exp4}significant: p-value {p4}")
print(f"Parents/Children is {exp5}significant: p-value {p5}")
print(f"Fare is {exp6}significant: p-value {p6}")



The response for x_new is 1
tau = 0.35390819464160644
the 95 percent confidence interval for omega_hat is 
 (0.16774488087208478,0.8755612701552977)
Passenger class is significant: p-value 0.0
Gender is significant: p-value 0.0
Age is significant: p-value 0.0
Siblings/Spouses is significant: p-value 3.314898653347422e-232
Parents/Children is significant: p-value 1.0355974960420728e-15
Fare is significant: p-value 0.0


In [268]:
# Another feature vector that we create ourselves
x_new2 = np.array([1, 0, 54, 1, 0, 71.2833,1])

print(f"The response for x_new2 is {response(theta_hat)(x_new2)}")
omega_hat2 = np.dot(theta_hat.T, x_new2)

mod_var2 = np.dot(x_new2.T, np.dot(fisher_inv, x_new2))  
                             
tau2 = norm.ppf(0.975, loc = 0, scale = sqrt(mod_var2) )
print(f"tau2 = {tau2}")
print(f"the 95 percent confidence interval for omega_hat2 is \n ({omega_hat2 - tau2},{omega_hat2 + tau2})")

The response for x_new2 is 0
tau2 = 0.4467018787426479
the 95 percent confidence interval for omega_hat2 is 
 (-1.613081222576111,-0.7196774650908152)
